# 1. Importing Libraries

In [166]:
import pandas as pd
import numpy as sns

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, accuracy_score, classification_report, precision_recall_fscore_support
from scipy.stats import chi2_contingency, f_oneway
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestClassifier

import os
import warnings

# 2. Reading Datasets 

In [5]:
a1 = pd.read_excel("data/case_study1.xlsx")
a2 = pd.read_excel("data/case_study2.xlsx")

In [180]:
df1 = a1.copy()
df2 = a2.copy()

In [181]:
df1.shape, df2.shape

((51336, 26), (51336, 62))

In [182]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51336 entries, 0 to 51335
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PROSPECTID            51336 non-null  int64  
 1   Total_TL              51336 non-null  int64  
 2   Tot_Closed_TL         51336 non-null  int64  
 3   Tot_Active_TL         51336 non-null  int64  
 4   Total_TL_opened_L6M   51336 non-null  int64  
 5   Tot_TL_closed_L6M     51336 non-null  int64  
 6   pct_tl_open_L6M       51336 non-null  float64
 7   pct_tl_closed_L6M     51336 non-null  float64
 8   pct_active_tl         51336 non-null  float64
 9   pct_closed_tl         51336 non-null  float64
 10  Total_TL_opened_L12M  51336 non-null  int64  
 11  Tot_TL_closed_L12M    51336 non-null  int64  
 12  pct_tl_open_L12M      51336 non-null  float64
 13  pct_tl_closed_L12M    51336 non-null  float64
 14  Tot_Missed_Pmnt       51336 non-null  int64  
 15  Auto_TL            

In [183]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51336 entries, 0 to 51335
Data columns (total 62 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PROSPECTID                    51336 non-null  int64  
 1   time_since_recent_payment     51336 non-null  int64  
 2   time_since_first_deliquency   51336 non-null  int64  
 3   time_since_recent_deliquency  51336 non-null  int64  
 4   num_times_delinquent          51336 non-null  int64  
 5   max_delinquency_level         51336 non-null  int64  
 6   max_recent_level_of_deliq     51336 non-null  int64  
 7   num_deliq_6mts                51336 non-null  int64  
 8   num_deliq_12mts               51336 non-null  int64  
 9   num_deliq_6_12mts             51336 non-null  int64  
 10  max_deliq_6mts                51336 non-null  int64  
 11  max_deliq_12mts               51336 non-null  int64  
 12  num_times_30p_dpd             51336 non-null  int64  
 13  n

# 3. Handling Null Values

In these DF null values are represented by -99999.<br>
Also we take following rules into consideration while dealing null values:
- Since this is a very sensitive data and we don't want to make any assumption about the missing values and don't want to impute them with some other values. Hence,
- If there is more than 20% data in a particular column that is missing, then we'll drop that column.
- If there is less than 20% missing data, we will remove those rows from the dataset.

In [184]:
# percent of Null data in each column of DF1
100*(df1 == -99999).mean()[(df1 == -99999).mean() > 0].sort_values(ascending=False)

Age_Oldest_TL    0.077918
Age_Newest_TL    0.077918
dtype: float64

In [185]:
# %tage of null data in each column of DF2
null_cols = (df2 == -99999).mean()[(df2 == -99999).mean()>0]*100
null_cols.sort_values(ascending=False)

CC_utilization                  92.792582
PL_utilization                  86.557192
time_since_recent_deliquency    70.026882
max_delinquency_level           70.026882
time_since_first_deliquency     70.026882
max_unsec_exposure_inPct        45.149603
max_deliq_6mts                  25.109085
max_deliq_12mts                 21.100203
time_since_recent_enq           12.312997
enq_L3m                         12.312997
enq_L6m                         12.312997
enq_L12m                        12.312997
PL_enq                          12.312997
PL_enq_L12m                     12.312997
PL_enq_L6m                      12.312997
CC_enq_L12m                     12.312997
CC_enq_L6m                      12.312997
CC_enq                          12.312997
tot_enq                         12.312997
time_since_recent_payment        8.358657
pct_currentBal_all_TL            0.140252
dtype: float64

In [186]:
# removing rows from first dataset as  null% < 20%

df1 = df1.loc[(df1["Age_Oldest_TL"] != -99999) & (df1 ["Age_Newest_TL"] != -99999)]

In [187]:
# removing columns from DF2 where null % > 20%

columns_to_drop = []
for col in df2.columns:
    if df2.loc[df2[col] == -99999].shape[0] > 10000:
        columns_to_drop.append(col)
columns_to_drop
df2.drop(columns=columns_to_drop, inplace=True)

In [188]:
# removing null rows from DF2 where null % < 20%

for col in df2.columns:
    df2 = df2.loc[df2[col] != -99999]

In [189]:
df1.isnull().sum().sum(),df2.isnull().sum().sum()

(0, 0)

# 4. Mergin two datasets 

In [190]:
# checking for common colums in DF1 and DF2
col1 = set(df1.columns)
col2 = set(df2.columns)
col1.intersection(col2)

{'PROSPECTID'}

In [191]:
# merging datasets on "PROSPECTID"
df = pd.merge(df1, df2, how="inner", on="PROSPECTID")

In [192]:
df

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,5,4,1,0,0,0.000,0.00,0.200,0.800,...,0.0,0.0,0.000,0.0,1,0,PL,PL,696,P2
1,2,1,0,1,0,0,0.000,0.00,1.000,0.000,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,8,0,8,1,0,0.125,0.00,1.000,0.000,...,0.0,0.0,0.000,0.0,1,0,ConsumerLoan,others,693,P2
3,5,3,2,1,0,0,0.000,0.00,0.333,0.667,...,0.0,0.0,0.000,0.0,0,0,AL,AL,753,P1
4,6,6,5,1,0,0,0.000,0.00,0.167,0.833,...,1.0,0.0,0.429,0.0,1,0,ConsumerLoan,PL,668,P3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42059,51332,3,0,3,1,0,0.333,0.00,1.000,0.000,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,650,P4
42060,51333,4,2,2,0,1,0.000,0.25,0.500,0.500,...,0.0,0.0,0.000,0.0,0,0,others,others,702,P1
42061,51334,2,1,1,1,1,0.500,0.50,0.500,0.500,...,1.0,0.0,1.000,0.0,0,0,ConsumerLoan,others,661,P3
42062,51335,2,1,1,0,0,0.000,0.00,0.500,0.500,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,others,686,P2


# 5. Features Selection: Categorical and Numerical Features 

## 5.1 Chi-square test for Categorical Features

In [193]:
categorical_cols = [col for col in df.columns if df[col].dtype == "O" and col != "Approved_Flag"]
categorical_cols

['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']

In [194]:
for col in categorical_cols:
    chi2, pvalue, dof, frequecies = chi2_contingency(pd.crosstab(df[col], df["Approved_Flag"]))
    print(col, "----->", pvalue)

MARITALSTATUS -----> 3.578180861038862e-233
EDUCATION -----> 2.6942265249737532e-30
GENDER -----> 1.907936100186563e-05
last_prod_enq2 -----> 0.0
first_prod_enq2 -----> 7.84997610555419e-287


Since all the p-values are less than 0.05, we accept alternate hypothesis that all these columns have significant importance in deciding our label "Approved_Flag"

## 5.2 Checking for Multicollinearity: VIF 

In [195]:
numerical_cols = [col for col in df.columns if df[col].dtype != "O" and col not in ["PROSPECTID"]]

In [196]:
def sequential_vif(df, threshold=6):
    columns_to_be_kept = []
    index = 0
    total_columns = df.shape[1]
    column_names = df.columns

    for i in range(0, total_columns):
        vif_score = variance_inflation_factor(df, index)

        if vif_score <= threshold:
            columns_to_be_kept.append(column_names[i])
            index += 1
        else:
            df = df.drop(columns=column_names[i])

    return columns_to_be_kept

In [197]:
vif_data = df[numerical_cols]
non_multicollinear_cols = sequential_vif(vif_data)

C:\Users\riyac\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
C:\Users\riyac\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
C:\Users\riyac\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
C:\Users\riyac\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
C:\Users\riyac\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered

## 5.3 One-Way ANNOVA for Numeric Feature Importance

In [198]:
numerical_columns_to_be_kept = [] 

for col in non_multicollinear_cols:
    a = list(df[col])
    b = list(df["Approved_Flag"])

    group_p1 = [value for value, group in zip(a,b) if group == "P1"]
    group_p2 = [value for value, group in zip(a,b) if group == "P2"]
    group_p3 = [value for value, group in zip(a,b) if group == "P3"]
    group_p4 = [value for value, group in zip(a,b) if group == "P4"]

    f_stats, p_value = f_oneway(group_p1, group_p2, group_p3,group_p4)

    if p_value <= 0.05:
        numerical_columns_to_be_kept.append(col)


In [199]:
numerical_columns_to_be_kept

['pct_tl_open_L6M',
 'pct_tl_closed_L6M',
 'Tot_TL_closed_L12M',
 'pct_tl_closed_L12M',
 'Tot_Missed_Pmnt',
 'CC_TL',
 'Home_TL',
 'PL_TL',
 'Secured_TL',
 'Unsecured_TL',
 'Other_TL',
 'Age_Oldest_TL',
 'Age_Newest_TL',
 'time_since_recent_payment',
 'max_recent_level_of_deliq',
 'num_deliq_6_12mts',
 'num_times_60p_dpd',
 'num_std_12mts',
 'num_sub',
 'num_sub_6mts',
 'num_sub_12mts',
 'num_dbt',
 'num_dbt_12mts',
 'num_lss',
 'recent_level_of_deliq',
 'CC_enq_L12m',
 'PL_enq_L12m',
 'time_since_recent_enq',
 'enq_L3m',
 'NETMONTHLYINCOME',
 'Time_With_Curr_Empr',
 'CC_Flag',
 'PL_Flag',
 'pct_PL_enq_L6m_of_ever',
 'pct_CC_enq_L6m_of_ever',
 'HL_Flag',
 'GL_Flag']

## 5.4 Selecting only important columns

In [200]:
features = categorical_cols + numerical_columns_to_be_kept + ["Approved_Flag"]
df = df[features].copy()

# 6. Encoding Categorical Columns 

In [201]:
for col in categorical_cols:
    print(col, "--->", df[col].unique())

MARITALSTATUS ---> ['Married' 'Single']
EDUCATION ---> ['12TH' 'GRADUATE' 'SSC' 'POST-GRADUATE' 'UNDER GRADUATE' 'OTHERS'
 'PROFESSIONAL']
GENDER ---> ['M' 'F']
last_prod_enq2 ---> ['PL' 'ConsumerLoan' 'AL' 'CC' 'others' 'HL']
first_prod_enq2 ---> ['PL' 'ConsumerLoan' 'others' 'AL' 'HL' 'CC']


- "EDUCATION" need to be Ordinal Encoded.
- For all other features, we can OneHotEncode them.
  

## 6.1 Ordinal Enconding 

Ordinal feature -- EDUCATION Ordinal Order
- SSC            : 1
- 12TH           : 2
- GRADUATE       : 3
- UNDER GRADUATE : 3
- POST-GRADUATE  : 4
- OTHERS         : 1
- PROFESSIONAL   : 3

In [202]:
replace_dict = {
    "SSC": 1,
    "12TH": 2,
    "GRADUATE": 3,
    "UNDER GRADUATE": 3,
    "POST-GRADUATE": 4,
    "OTHERS":1,
    "PROFESSIONAL": 3
}

df["EDUCATION"].replace(replace_dict, inplace=True)

## 6.2 OneHot Encoding

In [217]:
ohe_cols = ['MARITALSTATUS', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']
ohe = OneHotEncoder(sparse_output=False)
ohe_data = pd.DataFrame(data = ohe.fit_transform(df[ohe_cols]),
                        columns= ohe.get_feature_names_out())

df_encoded = df.drop(columns=ohe_cols)
df_encoded = pd.concat([ohe_data, df_encoded], axis=1)

In [224]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 55 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   MARITALSTATUS_Married         42064 non-null  float64
 1   MARITALSTATUS_Single          42064 non-null  float64
 2   GENDER_F                      42064 non-null  float64
 3   GENDER_M                      42064 non-null  float64
 4   last_prod_enq2_AL             42064 non-null  float64
 5   last_prod_enq2_CC             42064 non-null  float64
 6   last_prod_enq2_ConsumerLoan   42064 non-null  float64
 7   last_prod_enq2_HL             42064 non-null  float64
 8   last_prod_enq2_PL             42064 non-null  float64
 9   last_prod_enq2_others         42064 non-null  float64
 10  first_prod_enq2_AL            42064 non-null  float64
 11  first_prod_enq2_CC            42064 non-null  float64
 12  first_prod_enq2_ConsumerLoan  42064 non-null  float64
 13  f

In [225]:
df_encoded.head(2)

,MARITALSTATUS_Married,MARITALSTATUS_Single,GENDER_F,GENDER_M,last_prod_enq2_AL,last_prod_enq2_CC,last_prod_enq2_ConsumerLoan,last_prod_enq2_HL,last_prod_enq2_PL,last_prod_enq2_others,...,enq_L3m,NETMONTHLYINCOME,Time_With_Curr_Empr,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,Approved_Flag
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,51000,114,0,1,0.0,0.0,1,0,P2
1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0,19000,50,0,0,0.0,0.0,0,0,P2
